# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from random import sample

import re

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

# Import original data and preprocessed data

In [2]:
INPUT_DIR = r'C:\Users\ASUS\Desktop\Academic Work\Academic Research\DEMAND PATTERN\M5 Forecasting\Estimate the unit sales of Walmart retail goods\m5-forecasting-accuracy_unzip files'
sales_train_val = pd.read_csv(f'{INPUT_DIR}/sales_train_validation.csv')
calendar = pd.read_csv(f'{INPUT_DIR}/calendar.csv')

### Note: This Step is for the extraction of California State and Foods category

In [3]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'FOODS')]

In [4]:
INPUT_DIR_2 = r'C:\Users\ASUS\Desktop\Academic Work\Academic Research\DEMAND PATTERN\CRUNCH TIME (WEEKLY AGGREGATION)\California\Foods\Demand Pattern Categorization'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

**Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)**

In [5]:
sales_smooth

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
1612,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,3,0,0,1,...,0,2,0,4,1,1,0,1,1,0
1613,FOODS_1_002_CA_1_validation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,0,1,0,1,...,1,3,1,0,0,1,2,0,0,0
1614,FOODS_1_003_CA_1_validation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,3,0,2,1,1,0,1,0,1,0
1615,FOODS_1_004_CA_1_validation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1617,FOODS_1_006_CA_1_validation,FOODS_1_006,FOODS_1,FOODS,CA_1,CA,0,0,3,1,...,1,3,1,0,2,1,2,3,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12186,FOODS_3_818_CA_4_validation,FOODS_3_818,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,0,4,2,1,7,5,2,6,5,5
12187,FOODS_3_819_CA_4_validation,FOODS_3_819,FOODS_3,FOODS,CA_4,CA,4,4,3,1,...,0,2,0,1,1,1,1,0,0,0
12188,FOODS_3_820_CA_4_validation,FOODS_3_820,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,0,1,1,2,0,4,2,5,1,3
12190,FOODS_3_822_CA_4_validation,FOODS_3_822,FOODS_3,FOODS,CA_4,CA,1,3,2,0,...,2,0,0,1,0,1,1,0,1,2


# User-defined functions to calculate Metrics and Algorithm

In [6]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [7]:
def Croston_TSB(ts,extra_periods=1,alpha=0.4,beta=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), probability(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
# Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1/(1 + first_occurence)
    f[0] = p[0]*a[0]
                 
    # Create all the t+1 forecasts
    for t in range(0,cols): 
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = beta*(1) + (1-beta)*p[t]  
        else:
            a[t+1] = a[t]
            p[t+1] = (1-beta)*p[t]       
        f[t+1] = p[t+1]*a[t+1]
        
    # Future Forecast
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Format sales data and then merge with calendar data

In [8]:
sales_pattern = sales_smooth.copy()

In [9]:
sales_pattern_py = sales_pattern.copy()

list_pattern_py = sales_pattern_py.id.unique().tolist()
list_pattern_py = [(re.sub(r'_validation', r"", x)) for x in list_pattern_py]

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
df_pattern.reset_index(inplace=True)

calendar['Date'] = pd.to_datetime(calendar['date'])

dataframe = pd.merge(df_pattern,
                     calendar[['wm_yr_wk', 'Date']],
                     on = 'Date')

not_enough_seven_days = dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})[dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})['Date'] < 7].index.tolist()
dataframe = dataframe[~dataframe['wm_yr_wk'].isin(not_enough_seven_days)]

dataframe.set_index('wm_yr_wk', inplace=True)

dataframe.drop('Date', axis=1, inplace=True)

dataframe_weekly = dataframe.groupby('wm_yr_wk').sum().reset_index()

In [10]:
start_train_week = 11318
end_train_week = 11610
n_pred_weeks = 2
num_train_needed = 50
validation_training_ratio = 0.95
num_id_sample_for_tuning = 200

In [11]:
list_params_alpha = [round(item, 2) for item in list(np.arange(0.05, 0.95, 0.05))]
list_params_beta = [round(item, 2) for item in list(np.arange(0.05, 0.95, 0.05))]

In [12]:
len(list_pattern_py)

3594

In [13]:
sample_list = sample(list_pattern_py, num_id_sample_for_tuning)

dataframe_check = dataframe_weekly[sample_list].iloc[-10:-2, :].T
dataframe_check['Check'] = np.where((dataframe_check==0).all(axis=1), 1, 0)

sample_removed = dataframe_check[dataframe_check['Check'] == 1].index.tolist()
sample_list_filtered = [x for x in sample_list if (x not in sample_removed)]

In [14]:
len(sample_list_filtered)

150

# Hyperparameter tuning for Croston TSB

In [15]:
train_data = dataframe_weekly[(dataframe_weekly['wm_yr_wk'] >= start_train_week) & (dataframe_weekly['wm_yr_wk'] <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]

In [16]:
start = timer()

df_result_all_params = pd.DataFrame()

for product in sample_list_filtered:
    
    print('Currently Running: %s' % product)
    print('Progressing: {0} %'.format(round(sample_list_filtered.index(product) / len(sample_list_filtered) * 100, 2)))

    train_first = train_data_first_part[product]
    train_second = train_data_second_part[product]
    
    for i in list_params_alpha:
        for j in list_params_beta:
            predictions = list()
            history = [x for x in train_first[num_train_needed*-1:]]
            for t in train_second.index.tolist():
                yhat = Croston_TSB(history, extra_periods=1, alpha = i, beta = j)['Forecast'].iloc[-1]
                predictions.append(yhat)
                history.append(train_second[t])
            df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                           'Actual Data': train_second,
                                           'Forecast': predictions,
                                           'Alpha': [i for count_i in range(len(train_second))],
                                           'Beta': [j for count_j in range(len(train_second))]})
            df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)    
            
end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running: FOODS_2_103_CA_1
Progressing: 0.0 %
Currently Running: FOODS_1_018_CA_1
Progressing: 0.67 %
Currently Running: FOODS_2_342_CA_3
Progressing: 1.33 %
Currently Running: FOODS_1_054_CA_3
Progressing: 2.0 %
Currently Running: FOODS_3_717_CA_2
Progressing: 2.67 %
Currently Running: FOODS_1_218_CA_1
Progressing: 3.33 %
Currently Running: FOODS_3_196_CA_1
Progressing: 4.0 %
Currently Running: FOODS_3_158_CA_1
Progressing: 4.67 %
Currently Running: FOODS_3_586_CA_3
Progressing: 5.33 %
Currently Running: FOODS_3_139_CA_4
Progressing: 6.0 %
Currently Running: FOODS_3_406_CA_3
Progressing: 6.67 %
Currently Running: FOODS_2_332_CA_1
Progressing: 7.33 %
Currently Running: FOODS_3_268_CA_1
Progressing: 8.0 %
Currently Running: FOODS_1_103_CA_3
Progressing: 8.67 %
Currently Running: FOODS_3_204_CA_4
Progressing: 9.33 %
Currently Running: FOODS_3_479_CA_1
Progressing: 10.0 %
Currently Running: FOODS_3_369_CA_1
Progressing: 10.67 %
Currently Running: FOODS_3_536_CA_3
Progressing: 11.

Currently Running: FOODS_3_354_CA_4
Progressing: 96.67 %
Currently Running: FOODS_3_708_CA_4
Progressing: 97.33 %
Currently Running: FOODS_3_405_CA_2
Progressing: 98.0 %
Currently Running: FOODS_3_746_CA_2
Progressing: 98.67 %
Currently Running: FOODS_1_215_CA_1
Progressing: 99.33 %
This line of code took 6.200140411666667 minutes


In [18]:
df_result_all_params

,Product,Actual Data,Forecast,Alpha,Beta
0,FOODS_2_103_CA_1,1.0,6.256245,0.05,0.05
1,FOODS_2_103_CA_1,0.0,5.999970,0.05,0.05
2,FOODS_2_103_CA_1,0.0,5.699971,0.05,0.05
3,FOODS_2_103_CA_1,0.0,5.414973,0.05,0.05
4,FOODS_2_103_CA_1,4.0,5.144224,0.05,0.05
...,...,...,...,...,...
388795,FOODS_1_215_CA_1,23.0,27.218601,0.90,0.90
388796,FOODS_1_215_CA_1,14.0,23.421860,0.90,0.90
388797,FOODS_1_215_CA_1,12.0,14.942186,0.90,0.90
388798,FOODS_1_215_CA_1,12.0,12.294219,0.90,0.90


# Summay all metrics

In [19]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [20]:
start = timer()

df_result_metrics_all_params = df_result_all_params.groupby(['Product', 'Alpha', 'Beta']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 1.2792296583333334 minutes


In [21]:
df_result_metrics_all_params

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_012_CA_2,0.05,0.05,0.700890,17.2510,17.6080,16.0510,15.719953,19.931454
1,FOODS_1_012_CA_2,0.05,0.10,0.700890,17.2510,17.6080,16.0510,15.719953,19.931454
2,FOODS_1_012_CA_2,0.05,0.15,0.700890,17.2510,17.6080,16.0510,15.719953,19.931454
3,FOODS_1_012_CA_2,0.05,0.20,0.700890,17.2510,17.6080,16.0510,15.719953,19.931454
4,FOODS_1_012_CA_2,0.05,0.25,0.700890,17.2510,17.6080,16.0510,15.719953,19.931454
...,...,...,...,...,...,...,...,...,...
48595,FOODS_3_826_CA_4,0.90,0.70,0.974352,53.2565,58.1387,60.2219,6.124499,6.525090
48596,FOODS_3_826_CA_4,0.90,0.75,0.974352,53.2565,58.1387,60.2219,6.124499,6.525090
48597,FOODS_3_826_CA_4,0.90,0.80,0.974352,53.2565,58.1387,60.2219,6.124499,6.525090
48598,FOODS_3_826_CA_4,0.90,0.85,0.974352,53.2565,58.1387,60.2219,6.124499,6.525090


# Check what products has unexpected metrics result

In [22]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [23]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [24]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
33048,FOODS_3_437_CA_1,0.05,0.05,1.281158,94.1259,NaN,93.4310,4.941608,6.324751
33049,FOODS_3_437_CA_1,0.05,0.10,1.212829,89.1058,NaN,87.6608,4.678057,6.005847
33050,FOODS_3_437_CA_1,0.05,0.15,1.153192,84.7243,NaN,82.4534,4.448026,5.735132
33051,FOODS_3_437_CA_1,0.05,0.20,1.101675,80.9394,NaN,77.8001,4.249320,5.507450
33052,FOODS_3_437_CA_1,0.05,0.25,1.057313,77.6801,NaN,73.6490,4.078208,5.316485
...,...,...,...,...,...,...,...,...,...
42439,FOODS_3_677_CA_2,0.90,0.70,0.940625,86.0000,NaN,91.2500,0.537500,1.219119
42440,FOODS_3_677_CA_2,0.90,0.75,0.929688,85.0000,NaN,90.6250,0.531250,1.202212
42441,FOODS_3_677_CA_2,0.90,0.80,0.918750,84.0000,NaN,90.0000,0.525000,1.185327
42442,FOODS_3_677_CA_2,0.90,0.85,0.907813,83.0000,NaN,89.3750,0.518750,1.168466


In [25]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [26]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [27]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [28]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [29]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 0.0 %
Percentage of unexpected values of WMAPE is: 0.0 %
Percentage of unexpected values of SMAPE is: 2.666666666666667 %
Percentage of unexpected values of MAPE is: 0.0 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [30]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

# Get MEAN metrics value of each Alpha and Beta

In [31]:
df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

MASE      WMAPE      SMAPE       MAPE        MAE       RMSE
Alpha Beta                                                                 
0.05  0.05  1.207945  63.310895  57.871558  71.295734  13.916993  16.804076
      0.10  1.157842  60.025216  55.649342  69.835125  13.704021  16.600018
      0.15  1.127600  57.680301  54.886986  69.359690  13.605743  16.509325
      0.20  1.109660  56.212870  54.651904  69.286519  13.552504  16.453783
      0.25  1.095667  55.118027  54.484082  69.139317  13.511341  16.414756
...              ...        ...        ...        ...        ...        ...
0.90  0.70  0.978923  50.126096  56.259964  60.678815  12.894023  15.112939
      0.75  0.979046  50.163874  56.384338  60.751512  12.897516  15.114168
      0.80  0.979519  50.212692  56.549365  60.856886  12.901881  15.116173
      0.85  0.980232  50.273257  56.769673  60.997297  12.906901  15.118870
      0.90  0.980870  50.327187  57.000198  61.117366  12.911523  15.122189

[324 rows x 6 columns]

In [32]:
for mts in list_metrics:
    print('The optimum Alpha and Beta based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha and Beta based on Mean MASE is: (0.35, 0.9)
The optimum Alpha and Beta based on Mean WMAPE is: (0.35, 0.9)
The optimum Alpha and Beta based on Mean SMAPE is: (0.25, 0.5)
The optimum Alpha and Beta based on Mean MAPE is: (0.5, 0.1)
The optimum Alpha and Beta based on Mean MAE is: (0.45, 0.65)
The optimum Alpha and Beta based on Mean RMSE is: (0.6, 0.7)


# Get MEDIAN metrics value of each Alpha and Beta

In [33]:
df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

MASE     WMAPE     SMAPE      MAPE       MAE      RMSE
Alpha Beta                                                            
0.05  0.05  0.923186  44.63535  47.85750  50.48805  4.632454  5.268129
      0.10  0.906273  43.82610  45.57965  48.81245  4.353024  5.244178
      0.15  0.896091  42.90965  44.96800  48.06485  4.084203  5.244178
      0.20  0.896614  42.25060  44.96760  49.77365  4.074035  5.248945
      0.25  0.894901  42.07625  44.68210  49.78520  4.077261  5.238303
...              ...       ...       ...       ...       ...       ...
0.90  0.70  0.957075  45.87095  47.16290  48.89475  4.424632  5.417955
      0.75  0.958330  45.92845  47.16290  49.42620  4.424639  5.443530
      0.80  0.959624  45.97540  47.16290  49.50360  4.424641  5.464206
      0.85  0.964099  45.68340  47.16290  49.85895  4.424641  5.464207
      0.90  0.964502  45.29080  47.16290  50.31920  4.424641  5.464207

[324 rows x 6 columns]

In [34]:
for mts in list_metrics:
    print('The optimum Alpha and Beta based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha and Beta based on Median of MASE is: (0.15, 0.85)
The optimum Alpha and Beta based on Median of WMAPE is: (0.25, 0.65)
The optimum Alpha and Beta based on Median of SMAPE is: (0.25, 0.55)
The optimum Alpha and Beta based on Median of MAPE is: (0.15, 0.55)
The optimum Alpha and Beta based on Median of MAE is: (0.1, 0.85)
The optimum Alpha and Beta based on Median of RMSE is: (0.35, 0.75)


# Run Croston TSB model after decide best paramaters

In [35]:
best_alpha = 0.5
best_beta = 0.1

In [36]:
start = timer()

df_result_best_params = pd.DataFrame()
for product in list_pattern_py:
    train_product = train_data[product]
    test_product = test_data[product]
    predictions = list()
    history = [x for x in train_product[num_train_needed*-1:]]
    for t in test_product.index.tolist():
        yhat = Croston_TSB(history, extra_periods=1, alpha=best_alpha, beta = best_beta)['Forecast'].iloc[-1]
        predictions.append(yhat)
        history.append(test_product[t])
    df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                    'Actual Data': test_product,
                                    'Forecast': predictions})
    df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)
    
end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.08552530166666657 minutes


In [37]:
df_result_best_params

,Product,Actual Data,Forecast
0,FOODS_1_001_CA_1,8.0,6.535824
1,FOODS_1_001_CA_1,3.0,7.267912
2,FOODS_1_002_CA_1,3.0,0.569323
3,FOODS_1_002_CA_1,0.0,1.145407
4,FOODS_1_003_CA_1,8.0,10.983574
...,...,...,...
7183,FOODS_3_820_CA_4,5.0,3.475521
7184,FOODS_3_822_CA_4,18.0,17.264713
7185,FOODS_3_822_CA_4,18.0,17.632356
7186,FOODS_3_826_CA_4,13.0,10.985669


In [38]:
df_result_final = df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [39]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_001_CA_1,0.573209,52.1099,51.6384,80.2830,2.866044,3.190524
1,FOODS_1_001_CA_2,0.708412,25.0028,24.7470,26.6004,2.125236,2.157292
2,FOODS_1_001_CA_3,0.693942,113.5541,115.7502,376.6400,6.245474,6.288740
3,FOODS_1_001_CA_4,0.672922,134.5845,156.0127,71.8053,4.710456,4.721038
4,FOODS_1_002_CA_1,0.596014,119.2028,168.0991,81.0226,1.788042,1.900019
...,...,...,...,...,...,...,...
3589,FOODS_3_826_CA_1,0.765059,37.2191,39.9989,34.5775,6.885527,7.753699
3590,FOODS_3_826_CA_2,0.903587,129.0839,88.8326,411.8661,4.517936,5.751785
3591,FOODS_3_826_CA_4,0.774679,29.0505,32.1033,26.9101,4.648074,5.342395
3592,FOODS_3_827_CA_1,NaN,NaN,NaN,NaN,0.000000,0.000000


In [40]:
df_result_final.isnull().sum()

Product      0
MASE       711
WMAPE      711
SMAPE      712
MAPE       829
MAE          0
RMSE         0
dtype: int64

In [41]:
df_result_final.MAPE.mean()

59.994336925858946

In [42]:
df_result_final.to_csv('Croston_TSB_Smooth_Metrics.csv')